In [ ]:
import os
import json
import pprint
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import PIL
import cv2

In [ ]:
tfrecords_dir = 'tfrecords'
images_dir = '../input/hpa-single-label-cell-level-dataset/single-label-cell-level/rgb'
df = pd.read_csv('../input/hpasinglelabelcellcsv/singlelabelcellonly.csv')

In [ ]:
df.head()

In [ ]:
num_samples = 4096
num_tfrecords = len(df) // num_samples
if len(df) % num_samples:
    num_tfrecords += 1
    
if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)

num_tfrecords

In [ ]:
x= cv2.resize((cv2.imread( images_dir + '/' + '0f2aca4a-bbb6-11e8-b2ba-ac1f6b6435d0_12.png')), dsize= (224,224) )

In [ ]:
plt.imshow(x)

In [ ]:
def _bytes_feature(value, is_list=False):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    if not is_list:
        value = [value]
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value, is_list=False):
    """Returns a float_list from a float / double."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value, is_list=False):
    """Returns an int64_list from a bool / enum / int / uint."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

In [ ]:


def create_example(image, image_name, target):
    """
    Creates a tf.Example message ready to be written to a file from 4 features.

    Args:
        image (TBD): TBD
        image_name (str): TBD
        target (str): | delimited integers
    
    Returns:
        A tf.Example Message ready to be written to file
    """
    
    # Create a dictionary mapping the feature name to the 
    # tf.Example-compatible data type.
    feature = {
        'image': image_feature(((image))),
        'image_name': _bytes_feature(image_name, is_list=False),
        'target': _bytes_feature(target, is_list=False),
    }

    # Create a Features message using tf.train.Example.
    return tf.train.Example(features=tf.train.Features(feature=feature))
    

    



In [ ]:
# num_tfrecods = 64
# num_samples = len(df)
# N_EX_PER_REC = int(np.ceil(N_EX/64))
# write_tfrecords(train_ds.as_numpy_iterator(), out_dir="/kaggle/working/train_slide_records")


In [ ]:
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
for tfrec_num in tqdm(range(num_tfrecords)):
    samples = df[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]

    with tf.io.TFRecordWriter(
        tfrecords_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))
    ) as writer:
        for i in  tqdm(range(len(samples))): 
            image_name = samples.iloc[i].ID
            image_path = images_dir + '/' + image_name
            x = str(samples.iloc[i].Label).encode()
            image = tf.io.decode_jpeg(tf.io.read_file(image_path), channels = 3)  
            example = create_example(image, image_name.encode(), x)
            writer.write(example.SerializeToString())
            

In [ ]:
import pandas as pd
from glob import glob
import tensorflow as tf
from PIL import ImageFont
from typing import List, Tuple
from collections import Counter
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from kaggle_datasets import KaggleDatasets

In [ ]:
class TFRecordLoader:

    def __init__(self, image_size: List[int], n_classes: int, include_yellow_channel: bool):
        self.image_size = image_size
        self.n_classes = n_classes
        self.include_yellow_channel = include_yellow_channel

    def _parse_image(self, image):
        image = tf.image.decode_png(image, channels=3)
        image = tf.cast(image, dtype=tf.float32) / 255.0
        image = tf.image.resize(image, self.image_size)
        
        return image

    def _parse_label(self, label):
        indices = tf.strings.to_number(
            label       
        )
        indices =tf.cast(indices ,dtype = tf.uint8)
        return tf.one_hot(indices, depth=self.n_classes)
        

    def _make_example(self, example):
        feature_format = {
            'image': tf.io.FixedLenFeature([], dtype=tf.string),
            'image_name': tf.io.FixedLenFeature([], dtype=tf.string),
            'target': tf.io.FixedLenFeature([], dtype=tf.string)
        }
        features = tf.io.parse_single_example(example, features=feature_format)
        image = self._parse_image(features['image'])
        image_name = features['image_name']
        label = self._parse_label(features['target'])
        return image, image_name, label

    def _combine_channels(self, red, green, blue, yellow):
        # Ref: https://www.kaggle.com/dschettler8845/hpa-xai-ig-tfrecords-tpu-training
        (r_i, r_j, r_k), (g_i, g_j, g_k), (b_i, b_j, b_k), (y_i, y_j, y_k) = red, green, blue, yellow
        combined_image = tf.stack(
            [r_i[..., 0], g_i[..., 0], b_i[..., 0], y_i[..., 0]], axis=-1
        ) if self.include_yellow_channel else tf.stack(
            [r_i[..., 0], g_i[..., 0], b_i[..., 0]], axis=-1
        )
        return combined_image, r_k

    def _preprocess(self, dataset):
        # Ref: https://www.kaggle.com/dschettler8845/hpa-xai-ig-tfrecords-tpu-training
        red_dataset = dataset.filter(
            lambda x, y, z: tf.strings.regex_full_match(y, ".*red.*"))
        green_dataset = dataset.filter(
            lambda x, y, z: tf.strings.regex_full_match(y, ".*green.*"))
        blue_dataset = dataset.filter(
            lambda x, y, z: tf.strings.regex_full_match(y, ".*blue.*"))
        yellow_dataset = dataset.filter(
            lambda x, y, z: tf.strings.regex_full_match(y, ".*yellow.*"))
        dataset = tf.data.Dataset.zip(
            (red_dataset, green_dataset, blue_dataset, yellow_dataset)
        )
        dataset = dataset.map(
            map_func=self._combine_channels,
            num_parallel_calls=tf.data.AUTOTUNE
        )
        return dataset

    def get_dataset(self, train_tfrecord_files: List[str], ignore_order: bool = False):
        options = tf.data.Options()
        options.experimental_deterministic = False
        dataset = tf.data.TFRecordDataset(
            train_tfrecord_files, num_parallel_reads=tf.data.AUTOTUNE)
        dataset = dataset.with_options(options) if ignore_order else dataset
        dataset = dataset.map(
            map_func=self._make_example, num_parallel_calls=tf.data.AUTOTUNE)
        #dataset = self._preprocess(dataset)
        return dataset

In [ ]:
loader = TFRecordLoader(
    image_size=[224,224], n_classes=19, include_yellow_channel=False
)
dataset = loader.get_dataset('./tfrecords/file_00-4096.tfrec')

In [ ]:
for x in dataset.take(1
                     ):
    plt.imshow(x[0])
    print(x[0])